This chapter introduces **convolutional neural networks**, also known as convnets, a type of deep-learning model almost universally used in computer vision applications. You’ll learn to apply convnets to image-classification problems—in particular those involving small training datasets, which are the most common use case if you aren’t a large tech company.

The following lines of code show you what a basic convnet looks like. It’s a stack of `Conv2D` and `MaxPooling2D` layers. You’ll soon see exactly what they do.

In [1]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # Conv window 3x3, with 32 filters
model.add(layers.MaxPooling2D((2, 2))) # Max Pool window 2x2
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # Conv defaults with no stride
model.add(layers.MaxPooling2D((2, 2))) # Max Pool defaults with stride 2, thus downsampling by about 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Importantly, a convnet takes as input tensors of shape `(image_height, image_width, image_channels)` (not including the batch dimension). In this case, we’ll configure the convnet to process inputs of size `(28, 28, 1)`, which is the format of MNIST images.  
We’ll do this by passing the argument `input_shape=(28, 28, 1)` to the first layer.

Let’s display the architecture of the convnet so far:

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see that the output of every `Conv2D` and `MaxPooling2D` layer is a 3D tensor of shape `(height, width, channels)`. The width and height dimensions tend to shrink as you go deeper in the network. The number of channels is controlled by the first argument passed to the `Conv2D` layers (32 or 64).

The next step is to feed the last output tensor (of shape `(3, 3, 64)`) into a densely connected classifier network like those you’re already familiar with: **a stack of** `Dense` **layers**. These classifiers process vectors, which are 1D, whereas the current output is a 3D tensor. First we have to flatten the 3D outputs to 1D, and then add a few `Dense` layers on top.

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

We’ll do 10-way classification, using a final layer with 10 outputs and a softmax activation. Here’s what the network looks like now:

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

As you can see, the (3, 3, 64) outputs are flattened into vectors of shape `(576,)` before going through two `Dense` layers.

Now, let’s train the convnet on the MNIST digits.

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 13s 13ms/step - loss: 0.4094 - accuracy: 0.8681
Epoch 2/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0509 - accuracy: 0.9838
Epoch 3/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0335 - accuracy: 0.9896
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0236 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0167 - accuracy: 0.9946


In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 3ms/step - loss: 0.0268 - accuracy: 0.9920


0.9919999837875366

Whereas the densely connected network from chapter 2 had a test accuracy of about 97%, the basic convnet has a test accuracy of 99.3%: we decreased the error rate by 68% (relative). Not bad!

---

# The convolution operation
The fundamental difference between a densely connected layer and a *convolution* layer is this: `Dense` layers learn global patterns in their input feature space (for example, for a MNIST digit, patterns involving all pixels), whereas convolution layers learn local patterns: **in the case of images, patterns found in small 2D windows of the inputs**. In this previous example, these windows were all 3 × 3.

A convolution works by sliding these windows of size $3 \times 3$ or $5 \times 5$ (most commonly) over the 3D input feature map, stopping at every possible location, and extracting the 3D patch of surrounding features (shape `(window_height, window_width, input_depth)`).

Each such 3D patch is then transformed (via a tensor product with the same learned weight matrix, called the *convolution kernel*) into a 1D vector of shape `(output_depth,)`. All of these vectors are then spatially reassembled into a 3D output map of shape `(height, width, output_depth)`.

Note that the output width and height may differ from the input width and height. They may differ for two reasons:
- Border effects, which can be countered by padding the input feature map
- The use of strides

**Padding** consists of adding an appropriate number of rows and columns on each side of the input feature map so as to make it possible to fit center convolution windows around every input tile. For a $3 \times 3$ window, you add one column on the right, one column on the left, one row at the top, and one row at the bottom. For a $5 \times 5$ window, you add two rows.

**Strides** are the distance between successive windows. Using stride 2 means the width and height of the feature map are downsampled by a factor of 2 (in addition to any changes induced by border effects). Strided convolutions are rarely used in practice, although they can come in handy for some types of models; it’s good to be familiar with the concept. To downsample feature maps, instead of strides, we tend to use the *max-pooling operation*

# The max-pooling operation
In the convnet example above, you may notice that the size of the feature maps is halved after every `MaxPooling2D` layer. That’s the role of max pooling: to aggressively downsample feature maps, much like
strided convolutions.

**Max pooling** consists of extracting windows from the input feature maps and **outputting the max value** of each channel. It’s conceptually similar to convolution, except that instead of transforming local patches via a learned linear transformation (the convolution kernel), they’re transformed via a hardcoded `max` tensor operation.

A big difference from convolution is that max pooling is usually done with $2 \times 2$ windows and stride 2, in order to downsample the feature maps by a factor of 2. On the other hand, convolution is typically done with $3 \times 3$ windows and no stride (stride 1).

Why downsample feature maps this way? Why not remove the max-pooling layers and keep fairly large feature maps all the way up? Let’s look at this option. The convolutional base of the model would then look like this:

In [7]:
model_no_max_pool = models.Sequential()

model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [8]:
model_no_max_pool.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


What’s wrong with this setup? Two things:

-  It isn’t conducive to learning a spatial hierarchy of features. The $3 \times 3$ windows in the third layer will only contain information coming from $7 \times 7$ windows in the initial input. The high-level patterns learned by the convnet will still be very small with regard to the initial input, which may not be enough to learn to classify digits (try recognizing a digit by only looking at it through windows that are $7 \times 7$ pixels!). We need the features from the last convolution layer to contain information about the totality of the input.
- The final feature map has $22 \times 22 \times 64 = 30,976$ total coefficients per sample. This is huge. If you were to flatten it to stick a `Dense` layer of size $512$ on top, that layer would have $15.8$ million parameters. This is far too large for such a small model and would result in intense overfitting.

In short, the reason to use downsampling is to reduce the number of feature-map coefficients to process, as well as to induce spatial-filter hierarchies by making successive convolution layers look at increasingly large windows (in terms of the fraction of the original input they cover).

Note that max pooling isn’t the only way you can achieve such downsampling. As you already know, you can also use strides in the prior convolution layer. And you can use average pooling instead of max pooling, where each local input patch is transformed by taking the average value of each channel over the patch, rather than the max. But max pooling tends to work better than these alternative solutions.

In a nut-shell, the reason is that features tend to encode the spatial presence of some pattern or concept over the different tiles of the feature map (hence, the term *feature map*), and it’s more informative to look at the *maximal presence* of different features than at their *average presence*.

So the most reasonable subsampling strategy is to first produce dense maps of features (**via unstrided convolutions**) *and then* **look at the maximal activation of the features** over small patches, rather than looking at sparser windows of the inputs (via strided convolutions) or averaging input patches, which could cause you to miss or dilute feature-presence information.